## Pre-procesamiento de datos

* En este notebook veremos algunas herramientas y líneas de código para el preprocesamiento de datos para su posterior uso en Machine Learning

In [ ]:
# Importar librerías
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# Importar el dataset
df = pd.read_csv('00-datasets/Data.csv')

Se va a trabajar con un dataset de información salarial de personas con diferente edad y nacionalidad y el objetivo de este ejercicio sencillo es predecir si van a compar o no basados en los datos existentes

In [ ]:
df.head() # Observamos los primeros 5 registros (exploración de datos)

Podemos identificar que tenemos las variables X y Y, donde X son las variables independientes (País, Edad, Salario) y Y la variable dependiente que corresponde al campo Purchased, que será la variable objetivo para identificar si realizará o no una compra el cliente, basado en sus características (variables independientes) 

In [ ]:
X = df.iloc[:,:-1].values # Seleccionamos los campos y registros de las variables independientes excepto la columna Purchased (variable dependiente)
y = df.iloc[:,3].values # Seleccionamos la columna dependiente (Purchased). Otra opción: y = df.iloc[:,-1].values

Se observa que tenemos datos NaN que son datos no conocidos o faltantes, para ellos se pueden realizar varios procedimientos:
* Eliminar los registros (filas)
* Eliminar los campos (columnas)
* Imputación de valores

Cada caso depende del contexto de los datos y su naturaleza, para este caso específico, se realizará imputación de datos a través del valor promedio de todos los registros

In [ ]:
# Manejo de datos faltantes
from sklearn.impute import SimpleImputer # Importar el módulo necesario para la imputación datos faltantes (En este caso SimpleImputer de Scikit-Learn)
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')  # Crear una instancia de SimpleImputer con una estrategia para reemplazar los valores faltantes con la media
imputer.fit(X[:, 1:3]) # Se indica las columnas numéricas a las que le realizará la imputación (Age, Salary)
X[:, 1:3] = imputer.transform(X[:, 1:3]) # Transformar y reemplazar los valores faltantes en las columnas especificadas con la media
print(X) #Ver el resultado

El siguiente paso es analizar los datos categóricos, en este caso la columna Country
* El primer paso es codificar los datos categóricos de Country, debido a que para hacer una análisis se necesitan valores numéricos


In [ ]:
# Codificar datos categóricos (Este paso normalemente le llaman creacción de variables Dummy)
from sklearn.compose import ColumnTransformer # Importar el módulo necesario para la transormación de columnas (En este caso ColumnTransformer de Scikit-Learn)
from sklearn.preprocessing import OneHotEncoder # Importar el módulo necesario para la codificación (En este caso OneHotEncoder de Scikit-Learn)
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0])], remainder='passthrough') # Crear una instancia de ColumnTransformer con la especificación de la codificación One-Hot en la columna 0 y 'remainder' para mantener las demás columnas
X = np.array(ct.fit_transform(X)) # Transformar y codificar los datos utilizando la configuración definida en ColumnTransformer
print(X)

* El segundo paso es codificar la columna objetivo (Purchased), donde sus valores son Si o No

In [ ]:
# Codificar la variable Purchased (dependiente)
from sklearn.preprocessing import LabelEncoder # Importar el módulo necesario para la codificación (En este caso LabelEncoder de Scikit-Learn), nótese que como son solo dos valores, no es necesario OneHotEncoder
le = LabelEncoder() # Se crea una instancia para el codificador
y = le.fit_transform(y) # Aplicamos la transformación a la columna y (Purchased)
print(y)

Teniendo hecha la limpieza de los datos y su respectiva codifición, los siguientes pasos corresponden a la etapa de modelado, donde:
* Se realiza un split o division de los datos para obtener un porcentaje de datos a entrenar (train) y otro para evaluar (test) (Normalente se usa 80%-20%, 70%-30%, todo depende del contexto,naturaleza de los datos y experticia del profesional)
* Escalado de variables

In [ ]:

# Dividir el dataset en un conjuntos de entrenamiento (training) y otro de evaluación (testing)

from sklearn.model_selection import train_test_split # Importar el módulo necesario para la división (En este caso model_selection de Scikit-Learn).
#Se crean las cuatro variables para el train & test y se le indica la variables independientes (X) y la variable obejtico o dependiente (y).
# En test_size: 0.20 se está indicando que el 20% de los datos serán usados para el testing.

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1) # random_state: Se obliga al modelo que use la misma semilla, para evitar diferentes resultados cada vez que se ejecute.

# Mostramos las variables de train y test
print(X_train)
print(X_test)
print(y_train)
print(y_test)


Como se mencionó anteriormente, es necesario hacer una escalamiento de las variables a modelar, debido a que hay valores que no se podrían comparar debido a sus dimensiones y esto puede generar sesgos.

En este dataset, se observa que la edad maneja una dimension de 0 a 100 apróximadamente, mientras que el salario se encuentra en cientos de miles, por lo tanto cualquier análisis, se vería fuertemente inclinado por el tema salarial, sin importar la edad, es por eso que se suelen usar varias estrategias, dos de ellas son:
* Normalización, donde se ajustan los datos en una escala de 0 a 1.
* Estandarización, que depende de la media y sus desviación. -1 a 1

En este caso, usaremos la Estandarización, para trabajar una misma escala de medida

In [ ]:
# Escalado de variables
from sklearn.preprocessing import StandardScaler # # Importar el módulo necesario para la Estandarización (En este caso StandarScaler de Scikit-Learn).
sc = StandardScaler()  # Se crea una instancia para la estandarización
X_train[:, 3:] = sc.fit_transform(X_train[:, 3:]) # Se aplica la transformación a Age y Salary solamente, debido a que las otras variables están en rangos de 0 y 1
X_test[:, 3:] = sc.transform(X_test[:, 3:]) # Se aplica la transformación a X_test, nótese que no se usa fit, esto es porque queremos que aplique la misma transformación hecha a X_train
print(X_train)
print(X_test)